In [51]:
# imports
import pandas as pd
import numpy as np
import datetime
from dateutil import parser

In [13]:
# read all csvs into dataframes
dell_df = pd.read_csv('./datafiles/numerical-initial/dell-data.csv')
dow_df = pd.read_csv('./datafiles/numerical-initial/dow-data.csv')
hp_df = pd.read_csv('./datafiles/numerical-initial/hp-data.csv')
ibm_df = pd.read_csv('./datafiles/numerical-initial/ibm-data.csv')
nasdaq_df = pd.read_csv('./datafiles/numerical-initial/nasdaq-data.csv')
sandp500_df = pd.read_csv('./datafiles/numerical-initial/sandp500-data.csv')
seagate_df = pd.read_csv('./datafiles/numerical-initial/seagate-data.csv')
westdig_df = pd.read_csv('./datafiles/numerical-initial/westdig-data.csv')

In [14]:
# remove NaN rows
dell_df = dell_df.dropna(axis = 0, how = 'all')
dow_df = dell_df.dropna(axis = 0, how = 'all')
hp_df = dell_df.dropna(axis = 0, how = 'all')
ibm_df = dell_df.dropna(axis = 0, how = 'all')
nasdaq_df = dell_df.dropna(axis = 0, how = 'all')
sandp500_df = dell_df.dropna(axis = 0, how = 'all')
seagate_df = dell_df.dropna(axis = 0, how = 'all')
westdig_df = dell_df.dropna(axis = 0, how = 'all')

In [15]:
# function to convert volume o numerical
def df_cleanup(df):
    volumes = []
    for volume_str in df['Volume']:
        if volume_str.endswith('M'):
            volume_num = float(volume_str[0:len(volume_str)-1])*(10**6)
        elif volume_str.endswith('k'):
            volume_num = float(volume_str[0:len(volume_str)-1])*(10**3)
        volumes.append(volume_num)

        
    volume_df = pd.DataFrame({'Volume' : volumes})
    df['Volume'] = volume_df['Volume']
    df = df.drop(columns = ['SMAVG(15)'])
    return df

In [62]:
dell_df = df_cleanup(dell_df)
hp_df = df_cleanup(hp_df)
ibm_df = df_cleanup(ibm_df)
seagate_df = df_cleanup(seagate_df)
westdig_df = df_cleanup(westdig_df)

dow_df = df_cleanup(dow_df)
nasdaq_df = df_cleanup(nasdaq_df)
sandp500dig_df = df_cleanup(sandp500_df)

AttributeError: 'float' object has no attribute 'endswith'

In [94]:
dell_wsj_df = pd.read_csv('./scraper/dell_5yrs_wsj.csv')

In [95]:
def cleanup_wsj(df):
    df = df.drop([15,16])
    new_dates = []
    for date_str in df['date']:
        new_date_str = date_str.replace('Updated ', '')
        new_date_str = new_date_str.replace(' ET', '')
        new_date_str = new_date_str.replace(' | WSJ Pro', '')
        new_dates.append(parser.parse(new_date_str))
#         new_dates.append(new_date_str)
    date_df = pd.DataFrame({'date' : new_dates})
    df['date'] = date_df['date']
    
    return df

In [96]:
dell_wsj_df = cleanup_wsj(dell_wsj_df)
dell_wsj_df.to_csv('./data-cleanup/cleaned-dell-wsj.csv')